In [1]:
!pip install numpy
!pip install pandas
!pip install openpyxl
!pip install seaborn
!pip install matplotlib
!pip install -U scikit-learn
!pip install shap
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 832.5 kB/s eta 0:00:0031m1.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 33.3 MB/s eta 0:00:00m eta 0:00:010:01:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached pandas-2.2.1-cp310-cp310-macosx_10_9_x86_64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp310-cp310-macosx_10_9_x86_64.whl (12.5 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.2-py2.py3-none-an

In [2]:
import numpy as np
import pandas as pd
import openpyxl as xl
import FILibExcel
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb 
import shap

/Users/alexglushko/ProgProjects/ChemML/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data processing  

In [3]:
excel_path = r"../../static/mock/antibio_data/14.03.TZ_Arbanas.xlsx"
artifacts_path = r"../../static/mock/antibio_artifacts/"

In [5]:
tables_dict = FILibExcel.get_all_tables(file_name=excel_path)
df = tables_dict["Table1"]['dataframe']


worksheet name: Лист1
tables in worksheet: 1
table name: Table1


In [9]:
df.columns

Index(['RE', 'Образец', 'температура насыщения', 'обьем кристаллизатора. мл ',
       'время роста. ч', 'скорость перемешивания', 'плотность раствора. г/см3',
       'масса допанта ', 'масса исходной соли', 'масса воды. г',
       'концентрация допанта', 'сектор роста', 'масса кристалла', 'D1', 'D2',
       'D3', 'D4', 'D5', 'D6', 'D7', 'D8'],
      dtype='object')

In [10]:
X = df.drop(["RE", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8"], axis=1)
y = df[["D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8"]]
X.shape, y.shape

((14, 12), (14, 8))

# Learning

In [ ]:
def learn_xgboost(X, y):
    parameters = {
        'min_child_weight': [1, 5, 7, 10],
        'gamma': [0.5, 1, 1.5, 2, 2.5,],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
    }

    model = xgb.XGBRegressor(learning_rate=0.02, n_estimators=600, nthread=1, seed=0)
    clf = GridSearchCV(
        model, 
        parameters, 
        cv=5, 
        scoring='neg_mean_absolute_error',
        refit=True,
        n_jobs=-1)
    clf.fit(X, y)
    best_xgboost = clf.best_estimator_
    print(clf.best_score_)
    print(clf.best_params_)
    return best_xgboost